Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [23]:
from random import choices, sample, randint, random
import numpy as np
import lab9_lib
from copy import copy

In [24]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    individuals = np.array(choices([0, 1], k=50))
    print(f"{''.join(str(gene) for gene in individuals)}: {fitness(individuals):.2%}")

print(fitness.calls)

10101000110110000101001001011011100101111000110000: 7.33%
01101011111011000111010010001100111011011110101101: 9.11%
10101011101110001111001011011111000011011011011000: 31.34%
11001000000000011001100001001111011001010000001101: 7.36%
01100000111011111001001110010001110110111001001010: 15.33%
00011000000000001111011101000001000010110001011011: 23.56%
10110011010001111101011001110000011000011101101001: 23.34%
01111001000011101001101100101100000111111100001010: 7.33%
10001111101100100000101010110100100111101011011110: 23.33%
01011001111001010111110101100111000100101000010101: 23.33%
10


In [25]:
# I'm going to use genetic-algorithms
# I will therefore need crossover, mutation and selection

In [26]:
def selection(fitness_scores: np.ndarray, pop_size, tour_size):
    # choose k random individuals from the population and get their indexes
    candidate_idx = sample(range(0, pop_size), k=tour_size)
    # get the fitness scores of the respective candidates
    candidate_scores = [fitness_scores[idx] for idx in candidate_idx]
    # get the index (in population array) of the individual with the highest fitness
    best_candidate_idx = candidate_idx[np.argmax(candidate_scores)]

    return best_candidate_idx

In [77]:
def crossover(parent1: np.ndarray, parent2: np.ndarray, num_bits):

    # 1-cut crossover:
    # select random point on the genome to cut, but not on the ends of the genome, hence the 1 and the minus 2
    #cut_pt = randint(1, num_bits - 2)
    # use slicing to mix and match the genomes of the parents
    #child1 = np.concatenate((parent1[:cut_pt], parent2[cut_pt:]))
    #child2 = np.concatenate((parent2[:cut_pt], parent1[cut_pt:]))

    # this is the 2-cut crossover:
    #cut_pt1 = randint(1, num_bits - 3)
    #cut_pt2 = randint(cut_pt1 + 1, num_bits - 2)
    #child1 = np.concatenate((parent1[:cut_pt1], parent2[cut_pt1:cut_pt2], parent1[cut_pt2:]))
    #child2 = np.concatenate((parent2[:cut_pt1], parent1[cut_pt1:cut_pt2], parent2[cut_pt2:]))

    # this is the swapping of every other gene on the genome
    child1 = copy(parent1)
    child2 = copy(parent2)
    child1[0::2], child2[0::2] = parent2[0::2], parent1[0::2]

    # shifting the genes one spot to the right

    child1 = np.concatenate([child1[2:], child1[:2]])
    child2 = np.concatenate([child2[2:], child2[:2]])

    return [child1, child2]

p1 = np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
p2 = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

children = crossover(p1, p2, len(p1))

print("child 1: ", children[0])
print("child 2: ", children[1])

child 1:  [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
child 2:  [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]


In [61]:
def mutate(ind: np.ndarray, num_bits, mut_rate):
    mutated_ind = copy(ind)
    # for each bit in the individuals genome, mutate the gene with a prop = MUT_RATE
    for i in range(num_bits):
        if random() < mut_rate:
            mutated_ind[i] = not ind[i]
    
    return mutated_ind

In [62]:
def ga(fitness, t_size, mut_rate, pop_size, num_bits, max_gens):
    
    # first, initialize (pop_size) number of individuals with (num_bits) number of bits
    individuals = np.array([np.array(choices([0, 1], k=num_bits)) for _ in range(pop_size)])
    # calculate the initital fitness scores of the individuals
    fitness_scores = np.array([fitness(ind) for ind in individuals])

    # print("Initial average fitness: ", fitness_scores.sum() / pop_size)

    gen = 0

    while gen < max_gens and fitness_scores.max() < 1:
        parents = np.array([individuals[selection(fitness_scores, pop_size, t_size)] for _ in range(pop_size)])

        new_individuals = []

        for i in range(0, pop_size, 2):
            # take two and two individuals from the selection and crossover to make two new children
            parent1, parent2 = parents[i], parents[i+1]
            children = crossover(parent1, parent2, num_bits)

            # mutate the children with a probability of (mut_rate) for each bit in their genome
            for child in children:
                mutated_child = mutate(child, num_bits, mut_rate)
                new_individuals.append(mutated_child)

        if (len(new_individuals) != pop_size):
            print("wrong number of new individuals generated!: ", len(new_individuals))

        individuals = np.array(new_individuals)

        fitness_scores =  np.array([fitness(ind) for ind in individuals])

        gen += 1
    
    return individuals, fitness_scores, gen

In [89]:
TOURNAMENT_SIZE = 3
POP_SIZE = 200 # must be even number
NUM_BITS = 100
MUT_RATE = 1 / NUM_BITS
MAX_GENS = 10000

problem = lab9_lib.make_problem(10)

inds, fitness_scores, gens = ga(problem, TOURNAMENT_SIZE, MUT_RATE, POP_SIZE, NUM_BITS, MAX_GENS)

print("End average fitness: ", fitness_scores.sum() / POP_SIZE)

optimal = inds[np.argmax(fitness_scores)]

print(optimal)

if gens < MAX_GENS:
    print("\nOptimal solution found\n")
    optimal = inds[np.argmax(fitness_scores)]
    print(optimal)
    print("\nFound in num gens: ", gens)
    print("Number of fitness calls: ", problem.calls)

End average fitness:  0.49898039816615
[1 1 0 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1
 1 0 1 0 1 1 1 1 0 0 1 0 1 0 0 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0
 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 0]


In [10]:
# Now lets find the average number of fitness calls for each problem instance

TOURNAMENT_SIZE = 10
POP_SIZE = 10 # must be even number
NUM_BITS = 1000
MUT_RATE = 1 / NUM_BITS
MAX_GENS = 1000

problem_sizes = [1, 2, 5, 10]

for problem_size in problem_sizes:
    num_fitness_calls = []

    for i in range(10):
        problem = lab9_lib.make_problem(problem_size)
        inds, fitness_scores, gens = ga(problem, TOURNAMENT_SIZE, MUT_RATE, POP_SIZE, NUM_BITS, MAX_GENS)
        num_fitness_calls.append(problem.calls)
    
    average = np.array(num_fitness_calls).sum() / 100

    print(f"Average number of fitness calls for problem size ({problem_size}): {average}")
    

KeyboardInterrupt: 